
# 🧩 09 — MCP with Agents (Option A)

This notebook explains and sketches how to combine:

- **MCP (Model Context Protocol)** — your unified **capability layer**  
- **Agents** — from the Agents Universe

to build **tool-rich, safe, extensible agent systems**.

> Think of MCP as:  
> “One standardized way to expose tools (filesystem, HTTP, DB, RAG, etc.)  
> to any agent or LLM, regardless of framework.”



## 1. Why MCP + Agents?

Without MCP:

- Each agent system has its own way to define tools  
- Integrations are duplicated (filesystem, APIs, DBs, etc.)  
- Harder to share tools across:
  - RAG systems  
  - Agents  
  - IDEs  
  - CLIs  

With MCP:

- Tools are exposed via a **standard protocol**  
- Agents can **discover** and **call** these tools  
- You have a consistent, debuggable capability layer  

MCP + Agents makes it easier to:

- Build **multi-tool, multi-domain** agents  
- Reuse the **same tools** in:
  - RAG pipelines  
  - Agents  
  - UI assistants  
  - Dev tools  



## 2. Conceptual Architecture

At a high level:

- MCP **Servers** expose tools:
  - filesystem server
  - HTTP client server
  - DB server
  - RAG server
  - DevOps server
  - Finance or domain servers  

- MCP **Client** (inside your agent runtime) connects to one or more servers:
  - lists tools
  - calls tools
  - receives results  

- Agents treat MCP tools as **actions**:
  - `ACTION: mcp_call`
  - `ACTION_INPUT: { "server": "...", "tool": "...", "arguments": {...} }`



## 3. MCP Tool Call as a Generic Agent Action

We’ll sketch a generic **MCP tool call** action format:

```json
{
  "server": "fs_server",
  "tool": "read_file",
  "arguments": {
    "path": "docs/intro.md"
  }
}
```

The agent only needs to know:

- which **server**
- which **tool**
- which **arguments**

Everything else (network, protocol, implementation) is handled by:

- MCP server  
- MCP client library  



## 4. Python-Side Sketch of an MCP Tool Wrapper

Below is a **conceptual** example of wrapping MCP calls so that your agents can use them like normal tools.


In [ ]:
from typing import Dict, Any

# NOTE: This is a conceptual stub. In a real system:
# - You would use an MCP client library
# - Connect to one or more MCP servers
# - Perform actual network calls


class MCPClientStub:
    """Very simplified conceptual MCP client.

    In real code, you'd manage:
    - connections
    - request/response format
    - error handling
    """

    def call_tool(self, server: str, tool: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
        # TODO: Replace this with real MCP call logic.
        return {
            "server": server,
            "tool": tool,
            "arguments": arguments,
            "result": f"Mock result from MCP server '{server}', tool '{tool}'",
        }


mcp_client = MCPClientStub()


def mcp_tool_wrapper(input_dict: Dict[str, Any]) -> Dict[str, Any]:
    """Generic agent tool wrapper around MCP tools.

    Expected input_dict:
      {
        "server": "fs_server",
        "tool": "read_file",
        "arguments": { ... }
      }
    """
    server = input_dict.get("server")
    tool = input_dict.get("tool")
    arguments = input_dict.get("arguments", {}) or {}

    if not server or not tool:
        return {"error": "Missing 'server' or 'tool' in MCP call."}

    return mcp_client.call_tool(server=server, tool=tool, arguments=arguments)



### Registering `mcp_call` as an Agent Tool

We now plug `mcp_tool_wrapper` into the same `ToolRegistry` used in the Python hands-on agent.


In [ ]:
# Reuse ToolRegistry if present, otherwise define a minimal one.

try:
    registry  # type: ignore[name-defined]
except NameError:
    from dataclasses import dataclass
    from typing import Callable, List

    @dataclass
    class Tool:
        name: str
        description: str
        func: Callable[[Dict[str, Any]], Dict[str, Any]]

        def __call__(self, tool_input: Dict[str, Any]) -> Dict[str, Any]:
            return self.func(tool_input)

    class ToolRegistry:
        def __init__(self):
            self._tools: Dict[str, Tool] = {}

        def register(self, tool: Tool):
            self._tools[tool.name] = tool

        def get(self, name: str):
            return self._tools.get(name)

        def list_tools(self) -> List[Tool]:
            return list(self._tools.values())

    registry = ToolRegistry()

registry.register(Tool(
    name="mcp_call",
    description="Call an MCP server tool: {server, tool, arguments}.",
    func=mcp_tool_wrapper,
))



## 5. Prompting Agents to Use MCP Tools

We extend the agent’s **system prompt** so it knows:

- how to use `mcp_call`
- what arguments to provide  


In [ ]:
MCP_AGENT_SYSTEM_PROMPT = """You are an agent that can call MCP tools via the 'mcp_call' tool.

To use a tool on an MCP server, set:

ACTION: mcp_call
ACTION_INPUT: {
  "server": "<server_name>",
  "tool": "<tool_name>",
  "arguments": { ... }
}

Examples:

- Read a file from the filesystem server:
  ACTION: mcp_call
  ACTION_INPUT: {
    "server": "fs_server",
    "tool": "read_file",
    "arguments": { "path": "docs/intro.md" }
  }

- Query a RAG server:
  ACTION: mcp_call
  ACTION_INPUT: {
    "server": "rag_server",
    "tool": "search",
    "arguments": { "query": "agentic RAG", "top_k": 5 }
  }

You MUST respect allowed servers and tools documented to you.
"""


## 6. Example: Filesystem Agent via MCP

Conceptually, suppose you have an MCP filesystem server with tools:

- `list_dir(path)`
- `read_file(path)`

Your agent can:

1. List a directory of docs  
2. Read specific files  
3. Summarize or answer questions  

We’ll sketch a fake call:


In [ ]:
example_fs_call = {
    "server": "fs_server",
    "tool": "read_file",
    "arguments": {"path": "docs/intro.md"},
}

mcp_tool_wrapper(example_fs_call)



> 🔁 In your real implementation, the `MCPClientStub` will be replaced by
> your actual MCP client, which:
>
> - connects to the real MCP servers  
> - knows how to send/receive messages  
> - exposes a typed or generic tool-calling function  



## 7. Example: DevOps Agent via MCP

Suppose your MCP DevOps server exposes tools:

- `get_recent_logs(service, limit)`  
- `describe_incident(id)`  
- `suggest_mitigation(id)`  

Your agent could:

- respond to “What’s going on with service X?”  
- call logs + incident tools  
- summarize for the on-call engineer  

You prompt the agent (in natural language) + tell it about these tools.  
The agent uses `mcp_call` with `server: "devops_server"`, `tool: "get_recent_logs"`, etc.



## 8. Example: Finance Learning Agent via MCP

Combine with your **AI Finance** project:

- MCP Finance server:
  - `list_lessons()`
  - `get_lesson(id)`
  - `simulate_scenario(params)`  

Agent can:

- list content  
- pull specific lessons  
- simulate scenarios  
- then explain results in beginner-friendly language  

Again, the agent uses **only one action** conceptually:

- `ACTION: mcp_call`  
- `ACTION_INPUT: { "server": "finance_server", "tool": "simulate", ... }`



## 9. Multi-Agent + MCP: Manager Deciding Capabilities

You can mix patterns from the **Agentic Patterns notebook** with MCP:

- Manager agent:
  - decides which MCP tools to use  
  - selects which workers can access which servers  

- Worker agents:
  - restricted to certain servers:
    - e.g. logs-only, read-only DB, read-only finance  

Design principle:

> Limit **who** can call **which** MCP tools — don’t give every agent access to everything.



## 10. Safety Patterns for MCP + Agents

Key considerations:

- **Allow-list servers and tools**
  - per agent role  
- **Validate arguments** before calling MCP tools
  - types
  - ranges
  - safe paths (no `../` escapes)  

- **Rate-limit** critical tools
  - deployments
  - database writes  

- **Audit log** of:
  - which agent called which tool
  - with what arguments
  - what results were returned  

Combine this with:

- **step limits**
- **cost budgets**
- **human-in-the-loop approval** for sensitive actions



## 11. Where This Notebook Fits in Your Universes

This MCP + Agents notebook connects:

- **MCP Universe**
  - You design real MCP servers & tools  

- **Agents Universe**
  - You design agent loops, patterns, roles, memory  

- **RAG Universe**
  - You can expose your RAG pipelines behind an `rag_server` in MCP  

In the final notebook:

- `10_Full_RAG_Agents_MCP.ipynb`

you will combine:

- RAG tools  
- MCP capability layer  
- Agentic patterns  

into a **full, end-to-end architecture** for production-ready AI systems.
